In [2]:
import pandas as pd
import urllib
import json
from xpinyin import Pinyin
import re
import jieba
import time
from itertools import chain
from urllib import request
import numpy
from bs4 import BeautifulSoup as bs
import logging  # 引入logging模块
import os.path
import time

In [100]:
raw_data = pd.read_excel(u"定价系统训练数据_地区.xlsx")
pre_data = raw_data.loc[:, ["日期", "start_formatted_address","desti_formatted_address"]]

In [118]:
def deal_add(x):
    if isinstance(x, str):
        x = x[::-1]
#         get_shi_pattern = re.compile(u"\w{1,5}[省|自治区](\w{1,4})[市|回族自治州|蒙古自治州|地区|盟]\w+[区|旗|镇]\w+")
#         get_xianshi_pattern = re.compile(u"^\w{1,5}[省|自治区][\w{1,4}]*[市|回族自治州|蒙古自治州|地区|盟](\w{1,4})[县|市]\w*")
        get_xianshi_pattern = re.compile("\w+[县|市](\w{1,4})[市|州治自族回|州治自古蒙|区地|盟][\w{1,4}]*[省|区治自]\w{1,5}$")
        get_quxian_pattern = re.compile(u"^\w{1,3}市(\w{1,4})[区|县]")
        get_shi_pattern = re.compile(u"\w+[道街|区|旗|镇]\w{1,4}[市|州治自族回|州治自古蒙|区地|盟](\w{1,4})*[省|区治自]\w{1,5}$")
#         get_quxian_pattern = re.compile(u"^\w{1,3}市(\w{1,4})[区|县]")
#         get_shi_pattern1 = re.compile(u"^\w{1,5}[省|自治区](\w{1,5})*[市|回族自治州|蒙古自治州|地区|盟]\w{1,5}街道\w+$")

        if re.search(get_quxian_pattern, x):
            return re.search(get_quxian_pattern, x).groups()[0][::-1]   
        elif re.search(get_xianshi_pattern, x):
            return re.search(get_xianshi_pattern, x).groups()[0][::-1]
        elif re.search(get_shi_pattern, x):
            return re.search(get_shi_pattern, x).groups()[0][::-1]
        else:
            print(x)
            return None
    else:
        print(x)
        return None       

In [126]:
# 模糊查询             T[~T['A'].str.contains('我')]

In [78]:
get_xianshi_pattern = re.compile(u"^\w{1,5}[省|自治区][\w{1,4}]*[市|回族自治州|蒙古自治州|地区|盟](\w{1,4})[县|市]\w{2,10}$")
re.match(get_xianshi_pattern, "黑龙江省绥化市肇东市西园区街道肇东市政协").groups()


('街道肇东',)

In [80]:
s = "黑龙江省绥化市肇东市西园区街道肇东市政协"[::-1]
s

'协政市东肇道街区园西市东肇市化绥省江龙黑'

In [88]:
get_xianshi_pattern = re.compile("\w+[县|市](\w{1,4})[市|回族自治州|蒙古自治州|地区|盟][\w{1,4}]*[省|自治区]\w{1,5}$")
re.search(get_xianshi_pattern, s).groups()



('东肇',)

In [90]:
天津市河西区友谊路街道天津市委宣传部天津市人民政府
天津市河西区友谊路街道天津市委宣传部天津市人民政府
天津市津南区津南国家农业园区津南区政协

'府政民人市山中楼大府政市山中道街区东市山中省东广'

In [97]:
s = "天津市津南区津南国家农业园区津南区政协"[::-1]
get_quxian_pattern = re.compile(u"^\w{1,3}市(\w{1,4})[区|县]")
get_quxian_pattern = re.compile(u"\w+[区|县](\w{1,4})市\w{1,3}$")
re.search(get_quxian_pattern, s).groups()

('南津',)

In [95]:
s

'府政民人市津天部传宣委市津天道街路谊友区西河市津天'

In [93]:
s = "山东省泰安市岱岳区粥店街道泰安市岱岳区政协泰安市岱岳区人民政府"[::-1]

# get_shi_pattern = re.compile(u"^\w{1,5}[省|自治区](\w{1,5})*[市|回族自治州|蒙古自治州|地区|盟]\w{1,5}[|区|旗|镇]\w+$")
# get_shi_pattern1 = re.compile(u"^\w{1,5}[省|自治区](\w{1,5})*[市|回族自治州|蒙古自治州|地区|盟]\w{1,5}街道\w+$")

get_shi_pattern = re.compile(u"\w+[街道|区|旗|镇]\w{1,5}[市|回族自治州|蒙古自治州|地区|盟](\w{1,5})*[省|自治区]\w{1,5}")


re.search(get_shi_pattern, s).groups()

('安泰',)

In [30]:
"黑龙江省绥化市肇东市西园区街道肇东市政协"
"山东省泰安市岱岳区粥店街道泰安市岱岳区政协泰安市岱岳区人民政府"
"广东省中山市东区街道中山市政府大楼中山市人民政府"




'广东省中山市东区街道中山市政府大楼中山市人民政府'

In [124]:
xinjiang = pre_data[pre_data["start_formatted_address"].str.contains(u"广东")]
#xinjiang.loc[13217]

In [126]:
xinjiang["start_middle_loc"] = xinjiang["start_formatted_address"].map(deal_add)

C:\Anaconda2\envs\idp\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [198]:
get_xianshi_pattern = re.compile(u"^\w{1,7}[省|自治区]\w{1,4}[市|回族自治州|地区|盟](\w{1,4})[县|市]\w+")
re.search(get_xianshi_pattern, "广东省东莞市南城街道东莞市统计局东莞市人民政府").groups()[0]   

'丰镇'

In [62]:
raw_data["start_middle_loc"] = raw_data["start_formatted_address"].map(deal_add)

In [70]:
raw_data["desti_middle_loc"] = raw_data["desti_formatted_address"].map(deal_add)

[]


In [118]:
list(jieba.cut("新疆维吾尔自治区昌吉回族自治州阜康市城关镇阜康市阜康市人民医院(新院区)", cut_all = False))

['新疆维吾尔自治区',
 '昌吉回族自治州',
 '阜康市',
 '城关镇',
 '阜康市',
 '阜康市',
 '人民',
 '医院',
 '(',
 '新',
 '院区',
 ')']

In [71]:
writer = pd.ExcelWriter("pre_loc_data1.xlsx")
raw_data.to_excel(writer, "sheet1")
writer.save()

In [259]:
# 第一步，创建一个logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Log等级总开关
# 第二步，创建一个handler，用于写入日志文件
rq = time.strftime('%Y%m%d', time.localtime(time.time()))
log_path = os.path.dirname(os.getcwd()) + '/Logs/'
log_name = log_path + rq + '.log'
logfile = log_name
fh = logging.FileHandler(logfile, mode='w')
fh.setLevel(logging.DEBUG)  # 输出到file的log等级的开关
# 第三步，定义handler的输出格式
formatter = logging.Formatter("%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s")
fh.setFormatter(formatter)
# 第四步，将logger添加到handler里面
logger.addHandler(fh)


base_url = "https://lishi.tianqi.com/"
html = request.urlopen(base_url)
soup = bs(html, "html5lib")

def remove_blank(s):
    if isinstance(s, str):
        return s.strip()
    else:
        logger.debug("not str:",s)
        return None
    
    
def get_weather_temp(dates,loc, base_url):
    res = []
    dates_list = dates.split("-")
    year_mon = "".join(dates_list[:2])
    url = base_url.replace("index", year_mon)
    res.append(url)
    
    try:
        html = request.urlopen(url)
    except:
        html = None
    if html :
        soup = bs(html, "html5lib")
        if loc in soup.title.text:
            a_tag = soup.find("a", text = dates)
            if a_tag:
                ul = a_tag.parent.next_siblings
            else:
                ul = None
                logger.debug("find no a tag:%s"%url)

            if ul:
                for i in ul:
                    sub_res = remove_blank(i.string)
                    if sub_res is not None:
                        res.append(sub_res)
                return res
            
        else:
            return None
    else:
        logger.debug("获取网页失败:%s"%url)
        return None
    
    
def get_url(loc,soup = soup):
    #print(soup)
    if loc:
        target = soup.find("a", text = re.compile("^"+loc+"w*"))
        if target:
            return target.attrs["href"], loc
        else:
            return None, None
    else:
        return None, None

# target = soup.find("a", text = re.compile("^"+"河西"+"w*"))
# print(target)

In [246]:
html ="<li></li>"
bs(html).text is not None

C:\Anaconda2\envs\idp\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\idp\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


True

In [234]:
html = request.urlopen("http://lishi.tianqi.com/luanping/201704.html")
soup = bs(html)
'test' in soup.title.text

C:\Anaconda2\envs\idp\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\idp\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


False

In [3]:
raw_data = pd.read_excel("定价系统训练数据_地区_detail.xlsx")

In [4]:
raw_data.head()

,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,目的地纬度,运距,车型,车长,运价（元/吨）,start_formatted_address,desti_formatted_address,start_middle_loc,desti_middle_loc
0,2017-01-01,1,148,0,1,3333.33,114.203697,36.696506,114.670032,36.444317,57.90,19,NaN,27,河北省邯郸市武安市武安镇武安市人民政府,河北省邯郸市成安县成安镇河北省成安县政协,武安,成安
1,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,30.600064,1280.88,0,13.0,297,河北省唐山市玉田县玉田镇玉田县人大常委会,湖北省武汉市江岸区四唯街街道武汉市江岸区人民政府江岸区四唯街胜江社区,玉田,武汉
2,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,34.341568,1099.78,4,13.0,212,天津市河西区友谊路街道天津市委宣传部天津市人民政府,陕西省西安市未央区张家堡街道西安市林业局西安市人民政府,河西,西安
3,2017-01-01,2,225,0,1,8064.52,113.401679,35.099378,117.063754,30.543494,793.42,0,13.0,235,河南省焦作市武陟县木城街道兴华路8号,安徽省安庆市宜秀区菱北街道湖心北路84号安庆广播电视中心,武陟,安庆
4,2017-01-01,1,135,0,1,3672.87,118.402893,35.089917,118.909246,32.096228,391.41,0,13.0,122,山东省临沂市河东区九曲街道河东区,江苏省南京市栖霞区仙林街道南京市栖霞区人民政府,临沂,南京


In [4]:
###获取天气
start_middle_data = raw_data.loc[:, 
                                 ["日期", "start_district", "start_city", "start_province"]].rename(columns = {"日期":"日期", 
                                                                                                             "start_district":"district","start_city":"city",
                                                                                                            "start_province":"province"})
desti_middle_data = raw_data.loc[:, 
                                 ["日期", "desti_district", "desti_city", "desti_province"]].rename(columns = {"日期":"日期", 
                                                                                                             "desti_district":"district","desti_city":"city",
                                                                                                            "desti_province":"province"})

pre_data = pd.concat([start_middle_data, desti_middle_data], ignore_index = True)





In [5]:
len(pre_data)

427290

In [6]:
pre_data = pre_data.drop_duplicates()
pre_data = pre_data.dropna(how = "any")


In [135]:
writer = pd.ExcelWriter("detail_loc.xlsx")
pre_data.to_excel(writer, "sheet1")
writer.save()

In [3]:
pre_data = pd.read_excel("detail_loc.xlsx")

In [4]:
pre_data_values = pre_data.values

In [269]:
def get_weather(values):
    dates, district, city, province =values[0], values[1], values[2], values[3]
    if district !="[]":
        if len(district)>2:
            district = district[:-1]
            base_url, loc = get_url(district)
        else:
            base_url, loc = get_url(district)
    else:
        base_url = None
    if not base_url:
        if city !="[]":
            if len(city)>2:
                city = city[:-1]
                base_url,loc = get_url(city)
            else:
                base_url,loc = get_url(city)
        else:
            if len(province)>2:
                province = province[:-1]
                base_url,loc = get_url(province)
            else:
                base_url,loc = get_url(province)

    if base_url:
        res = get_weather_temp(dates, loc, base_url)
        if res:
            print(res)
            values = list(values)
            values.extend(res)
            return values

In [27]:
res = (get_weather(i) for i in pre_data_values)

In [7]:
len(pre_data_values)

73195

In [262]:
second_crawl_values = second_crawl.values

In [264]:
len(second_crawl_values)

43696

In [273]:
count = 42740
f = open("second_wether_info_1.txt", "a", encoding="utf-8")
for i in second_crawl_values[42740:]:
    try:
        sub_res = get_weather(i)
    except:
         sub_res = None
    f.writelines(str(sub_res)+'\n')
    count += 1
    print(count)
f.close()

['http://lishi.tianqi.com/pingshanqu/201711.html', '', '-6', '', '-14', '', '晴', '', '东风', '', '2级', '']
42741
['http://lishi.tianqi.com/sanmenxia/201711.html', '', '6', '', '-2', '', '雨夹雪', '', '东北风', '', '2级', '']
42742
['http://lishi.tianqi.com/xinglongtaiqu/201711.html', '', '-3', '', '-10', '', '晴', '', '东南风', '', '1级', '']
42743
['http://lishi.tianqi.com/lianyungang/201711.html', '', '9', '', '2', '', '阴', '', '东北风', '', '2级', '']
42744
['http://lishi.tianqi.com/huinong/201711.html', '', '4', '', '-10', '', '晴', '', '北风', '', '2级', '']
42745
['http://lishi.tianqi.com/pingshanqu/201711.html', '', '-7', '', '-15', '', '晴', '', '东北风', '', '1级', '']
42746
['http://lishi.tianqi.com/yongnian/201711.html', '', '4', '', '-4', '', '晴', '', '东南风', '', '1级', '']
42747
['http://lishi.tianqi.com/guannan/201711.html', '', '10', '', '-2', '', '多云', '', '东风', '', '微风', '']
42748
42749
['http://lishi.tianqi.com/zunhua/201711.html', '', '2', '', '-7', '', '晴', '', '西风', '', '1级', '']
42750
['http:

In [270]:
f.close()

In [ ]:
weather_df = pd.DataFrame(res, columns = ["日期","district", "city", "province", "weather_url", "high_temp", "low_temp", "weather", "wind_dire", "wind_degree"])

In [ ]:
writer = pd.ExcelWriter("weather_data.xlsx")
weather.to_excel(writer, "sheet1")
writer.save()

In [7]:
pre_data["base_url"] = pre_data["middle_loc"].map(get_url)

In [8]:
writer = pd.ExcelWriter("pre_data.xlsx")
pre_data.to_excel(writer, "sheet1")
writer.save()

In [216]:

base_url = "https://lishi.tianqi.com/"
html = request.urlopen(base_url)
soup = bs(html, "html5lib")
test = soup.find("a", text = re.compile("^"+"河西"+"w*"))
baseurl = test.attrs["href"]
baseurl.replace("index", "230")

'http://lishi.tianqi.com/hexiqu/230.html'

In [212]:
li = []
if li :
    print("s")

In [100]:
test

['2017-01-01', 'wuanshi', 1, 2, 3]

In [51]:
"2017-01-01".split("-")

['2017', '01', '01']

In [54]:
base_url = "http://lishi.tianqi.com/{}/{}.html"
base_url.format("sd", "asd")

'http://lishi.tianqi.com/sd/asd.html'

In [58]:
from urllib import request
import numpy
from bs4 import BeautifulSoup as bs 

In [85]:
html = request.urlopen("http://lishi.tianqi.com/wuan/201709.html")
soup = bs(html, "html5lib")
ul = soup.find("a", text = "2017-09-01").parent.next_siblings

In [86]:
test = list(ul)

In [87]:
for i in test:
    print(i.string)


		   			   	
26

		   	
18

		   	
多云

		   	
西南风

		   	
2级

    	   


In [278]:
res = []
middle_res = []
with open("second_wether_info_1.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        try:
            sub_res = eval(line)
        except:
            middle_res.append(line)
            sub_res = None
        if sub_res:
            mid_list = []
            for i in sub_res:
                if i != "":
                    mid_list.append(i)
            
            res.append(mid_list)

In [279]:
res[:10]

[['2017-01-01',
  '江岸区',
  '武汉市',
  '湖北省',
  'http://lishi.tianqi.com/jianganqu/201701.html',
  '13',
  '4',
  '多云',
  '北风',
  '微风'],
 ['2017-01-01',
  '宜秀区',
  '安庆市',
  '安徽省',
  'http://lishi.tianqi.com/yixiuqu/201701.html',
  '15',
  '4',
  '多云',
  '东南风',
  '1级'],
 ['2017-01-01',
  '萧山区',
  '杭州市',
  '浙江省',
  'http://lishi.tianqi.com/xiaoshan/201701.html',
  '16',
  '8',
  '晴',
  '北风',
  '3级'],
 ['2017-01-01',
  '中原区',
  '郑州市',
  '河南省',
  'http://lishi.tianqi.com/zhongyuanqu/201701.html',
  '7',
  '0',
  '霾',
  '西风',
  '1级'],
 ['2017-01-01',
  '宿城区',
  '宿迁市',
  '江苏省',
  'http://lishi.tianqi.com/suchengqu/201701.html',
  '12',
  '4',
  '多云',
  '东风',
  '1级'],
 ['2017-01-01',
  '东湖区',
  '南昌市',
  '江西省',
  'http://lishi.tianqi.com/donghuqu/201701.html',
  '16',
  '9',
  '晴',
  '北风',
  '微风'],
 ['2017-01-01',
  '长安区',
  '石家庄市',
  '河北省',
  'http://lishi.tianqi.com/changanqu/201701.html',
  '5',
  '-2',
  '霾',
  '西北风',
  '2级'],
 ['2017-01-01',
  '东城区',
  '[]',
  '北京市',
  'http://lishi.tianqi.c

In [ ]:
pd.DataFrame(res, columns = [""])

In [275]:
len(res)

39625

In [12]:
middle_res

["['2017-07-08', '任丘市', '沧州市', '河北省', 'http://lishi.tianqi.com/renqiu/201707.html', '38', '27', '晴', '南风['2017-04-16', '迁安市', '唐山市', '河北省', 'http://lishi.tianqi.com/qianan1/201704.html', '26', '15', '多云', '东南风', '2级']\n",
 "['2017-07-09', '鄂托克旗', '鄂尔多斯市', '内蒙古自治区', 'http://lishi.tianqi.com/etuoke/201707.html', '35'', '4', '阴', '西南风', '4级']\n",
 "['2017-04-24', '河东区', '临沂市', '山东省', 'http://lishi.tianqi.com/hedongqu/201704.html', '24', '12', '多云', '东南, '16', '多云', '东北风', '2级']\n",
 "['2017-07-10', '邯山区', '邯郸市', '河北省', 'http://lishi.tianqi.com/hanshanqu/201707.html', 风', '3级']\n",
 "['2017-04-29', '西青区', '[]', '天津市', 'http://lishi.tianqi.com/xiqing/201704.html', '33', '18', '晴', '西南风', '3级'37', '27', '晴', '南风', '2级']\n",
 "['2017-05-26', '丰南区', '唐山市', '河北省'级']\n",
 "['2017-07-14', '涞源县', '保定市, 'http://lishi.tianqi.com/fengnan/201705.html', '28', '18', '晴', '北风', '1级']\n",
 "['2017-06-01', '嘉祥县', '济宁市', '山东省', 'http://lishi.tianqi.com/jiaxiang/201706.html', '36',', '河北省', 'http://lishi.tia

In [16]:
def clean_string(strings):
    strings = strings.strip()
    try:
        str_list = eval(strings)
    except:
        str_list = None
    if str_list:
        return str_list
        

In [17]:
clean_res = []
for i in middle_res:
    clean_res.append(clean_string(i))

In [19]:
['2017-07-08', '任丘市', '沧州市', '河北省', 'http://lishi.tianqi.com/renqiu/201707.html', '38', '27', '晴', '南风', '2级']
['2017-04-16', '迁安市', '唐山市', '河北省', 'http://lishi.tianqi.com/qianan1/201704.html', '26', '15', '多云', '东南风', '2级']

['2017-04-16',
 '迁安市',
 '唐山市',
 '河北省',
 'http://lishi.tianqi.com/qianan1/201704.html',
 '26',
 '15',
 '多云',
 '东南风',
 '2级']

In [22]:
['2017-04-24', '河东区', '临沂市', '山东省', 'http://lishi.tianqi.com/hedongqu/201704.html', '24', '12', '多云', '东南','16', '多云', '东北风', '2级']

['2017-07-09', '鄂托克旗', '鄂尔多斯市', '内蒙古自治区', 'http://lishi.tianqi.com/etuoke/201707.html', '35', '4', '阴', '西南风', '4级']

['2017-07-09',
 '鄂托克旗',
 '鄂尔多斯市',
 '内蒙古自治区',
 'http://lishi.tianqi.com/etuoke/201707.html',
 '35',
 '4',
 '阴',
 '西南风',
 '4级']

In [27]:
['2017-07-10', '邯山区', '邯郸市', '河北省', 'http://lishi.tianqi.com/hanshanqu/201707.html', '37','27','晴','南风','2级']
['2017-04-29', '西青区', '[]', '天津市', 'http://lishi.tianqi.com/xiqing/201704.html', '33', '18', '晴', '西南风', '3级']
['2017-07-14', '涞源县', '保定市', '河北省', 'http://lishi.tianqi.com/laiyuan/201707.html', '34', '19', '中雨', '东北风', '1级']
['2017-06-01', '嘉祥县', '济宁市', '山东省', 'http://lishi.tianqi.com/jiaxiang/201706.html', '36', '20','多云','北风','3级']
['2017-06-06', '耀州区', '铜川市', '陕西省', 'http://lishi.tianqi.com/yaozhou/201706.html','25','15','晴','东北风','5级']
['2017-07-12','六合区', '南京市', '江苏省', 'http://lishi.tianqi.com/luhe1/201707.html', '27', '0', '多云', '南风', '小于3级']
['2017-07-14','六合区', '南京市', '江苏省', 'http://lishi.tianqi.com/luhe1/201707.html', '27', '0', '多云', '南风', '小于3级']

['2017-04-24', '河东区', '临沂市', '山东省', 'http://lishi.tianqi.com/hedongqu/201704.html', '24', '12', '多云', '东南','16', '多云', '东北风', '2级']

['2017-04-24',
 '河东区',
 '临沂市',
 '山东省',
 'http://lishi.tianqi.com/hedongqu/201704.html',
 '24',
 '12',
 '多云',
 '东南',
 '16',
 '多云',
 '东北风',
 '2级']

In [90]:
test_1 = [['2017-07-17', '溧阳市', '常州市', '江苏省', 'http://lishi.tianqi.com/liyang/201707.html','38','29','多云','东南风','1级']
,['2017-06-23', '成安县', '邯郸市', '河北省', 'http://lishi.tianqi.com/chengan/201706.html', '27','19','雷阵雨','东北风','3级']
,['2017-07-18', '襄汾县', '临汾市', '山西省', 'http://lishi.tianqi.com/xiangfen/201707.html', '34', '25', '小雨', '东南风','2级']

,['2017-06-27', '达拉特旗', '鄂尔多斯市', '内蒙古自治区', 'http://lishi.tianqi.com/dalate/201706.html','35','20','多云','东北风','2级']  

,['2017-07-03', '旬邑县', '咸阳市', '陕西省', 'http://lishi.tianqi.com/xunyi/201707.html','31','16','晴','南风','2级']
,['2017-07-20', '鄂托克旗', '鄂尔多斯市', '内蒙古自治区', 'http://lishi.tianqi.com/etuoke/201707.html', '33','22','多云','东北风','3级']

,['2017-07-18', '鄂托克旗', '鄂尔多斯市', '内蒙古自治区', 'http://lishi.tianqi.com/etuoke/201707.html', '32', '19', '多云', '东南风', '1级']
,['2017-06-18','惠济区','郑州市','河南省','http://lishi.tianqi.com/huijiqu/201706.html', '32', '23', '多云', '东风', '2级']]

In [91]:
test_2 = [['2017-05-26', '丰南区', '唐山市', '河北省','http://lishi.tianqi.com/fengnan/201705.html', '28', '18', '晴', '北风', '1级']
,['2017-07-19', '文峰区', '安阳市', '河南省', 'http://lishi.tianqi.com/wenfengqu/201707.html', '34', '25', '晴', '西北风','2级']] 

In [92]:
test_3 = [['2017-07-20', '泽州县', '晋城市', '山西省', 'http://lishi.tianqi.com/zezhou/201707.html', '33', '24', '晴', '东南风','2级']
,['2017-07-04', '玉田县', '唐山市', '河北省', 'http://lishi.tianqi.com/yutian/201707.html', '31', '23', '多云', '西北风', '1级']
,['2017-07-07', '西峡县', '南阳市', '河南省', 'http://lishi.tianqi.com/xixia/201707.html', '33', '25', '多云', '东风', '2级']

,['2017-07-12', '黄岛区', '青岛市', '山东省', 'http://lishi.tianqi.com/huangdao/201707.html','30','26','晴','东南风','2级']


,['2017-07-21', '榆阳区', '榆林市','陕西省', 'http://lishi.tianqi.com/yuyangqu/201707.html','32','22','小雨','东南风','3级']
,['2017-07-19', '泰山区', '泰安市', '山东省', 'http://lishi.tianqi.com/taishanqu/201707.html','33','26','阴南风','1级']
,['2017-08-01', '竞秀区', '保定市', '河北省', 'http://lishi.tianqi.com/baoding/201708.html', '30','23','多云','西北风','2级']
,['2017-07-24','离石区', '吕梁市', '山西省', 'http://lishi.tianqi.com/lishi1/201707.html', '32', '26', '阵雨', '东北风', '1级']
,['2017-07-24', '西工区', '洛阳市', '河南省', 'http://lishi.tianqi.com/xigongqu/201707.html','39','27','晴','东北风','4级']
,['2017-08-05', '丰润区', '唐山市', '河北省', 'http://lishi.tianqi.com/fengrun/201708.html', '34', '24','多云','西北风','2级']

,['2017-07-26', '武安市', '邯郸市', '河北省', 'http://lishi.tianqi.com/wuan/201707.html', '31', '21', '雷阵雨','东北风','3级']


,['2017-08-12', '博兴县', '滨州市', '山东省', 'http://lishi.tianqi.com/boxing/201708.html','34','24','多云','东风','2级']
,['2017-07-29', '西青区', '[]', '天津市', 'http://lishi.tianqi.com/xiqing/201707.html', '30', '22', '阴', '西南风', '2级']

,['2017-08-26','钢城区','莱芜市','山东省', 'http://lishi.tianqi.com/laiwu/201708.html', '30', '21', '多云', '北风', '微风']
,['2017-08-26','莱城区','莱芜市','山东省', 'http://lishi.tianqi.com/laiwu/201708.html', '30', '21', '多云', '北风', '微风']
,['2017-08-30', '溧阳市', '常州市', '江苏省', 'http://lishi.tianqi.com/liyang/201708.html','27','23','阴','东北风','2级']

,['2017-09-28', '安阳县','安阳市','河南省', 'http://lishi.tianqi.com/anyang/201709.html','25','16','晴','东南风','3级']
,['2017-08-04','鹤山区', '鹤壁市','河南省','http://lishi.tianqi.com/heshanqu1/201708.html', '36', '25', '晴','','1级']

,['2017-08-24','西工区','洛阳市','河南省','http://lishi.tianqi.com/xigongqu/201707.html', '39', '27', '晴', '东北风', '4级']


,['2017-08-05', '惠济区', '郑州市', '河南省', 'http://lishi.tianqi.com/huijiqu/201708.html', '37', '26', '晴', '东风', '1级']


,['2017-10-09','都江堰市', '成都市', '四川省', 'http://lishi.tianqi.com/dujiangyan/201710.html', '19', '14', '阴', '东南风', '2级']
,['2017-10-13', '寿光市', '潍坊市', '山东省', 'http://lishi.tianqi.com/shouguang/201710.html','20','12','多云','东风','1级']
,['2017-11-20', '东台市', '盐城市', '江苏省', 'http://lishi.tianqi.com/dongtai/201711.html', '11', '2', '多云','西北风','1级']
,['2017-08-13','当涂县', '马鞍山市', '安徽省', 'http://lishi.tianqi.com/dangtu/201708.html', '30', '25', '中雨', '西南风', '1级']
,['2017-12-01', '临城县', '邢台市', '河北省','http://lishi.tianqi.com/lincheng/201712.html','7','-5','晴','南风','3级']
,['2017-12-05', '邯山区', '邯郸市', '河北省', 'http://lishi.tianqi.com/hanshanqu/201712.html', '9','-3','晴','南风','2级']
,['2017-08-15','辛集市','石家庄市','河北省','http://lishi.tianqi.com/xinji/201708.html', '32', '24', '晴', '北风', '2级']

,['2017-08-16', '马村区', '焦作市', '河南省', 'http://lishi.tianqi.com/xinji/201708.html','34','23','多云','北风','1级']
,['2017-08-18', '武侯区', '成都市', '四川省','http://lishi.tianqi.com/wuhouqu/201708.html', '29','23','小雨','北风','2级']

,['2017-08-19', '滨城区', '滨州市', '山东省','http://lishi.tianqi.com/binxianqu/201708.html','28','23','雷阵雨','东北风','2级']
,['2017-12-23','安泽县','临汾市','山西省', 'http://lishi.tianqi.com/anze/201712.html', '7', '-8', '阴', '西风', '2级']
,['2017-12-30', '成安县','邯郸市','河北省', 'http://lishi.tianqi.com/chengan/201712.html', '8','-5','晴','北风','微风']
,['2017-08-20', '泽州县', '晋城市', '山西省', 'http://lishi.tianqi.com/zezhou/201708.html', '30', '20', '多云', '东南风', '1级']

,['2017-08-21', '云龙区', '徐州市', '江苏省', 'http://lishi.tianqi.com/yunlongqu/201708.html','32','23','多云','东风','2级']
,['2017-12-26', '乌拉特中旗','巴彦淖尔市','内蒙古自治区', 'http://lishi.tianqi.com/bayanzhuoer/201712.html', '1', '-12', '晴', '北风', '2级']
,['2017-01-11', '齐河县', '德州市', '山东省', 'http://lishi.tianqi.com/qihe/201701.html', '7', '-5', '多云','南风','2级']
,['2017-08-21','金台区','宝鸡市','陕西省','http://lishi.tianqi.com/jintaiqu/201708.html', '26', '20', '小雨', '西南风', '1级']
,['2017-02-04', '宝安区', '深圳市', '广东省', 'http://lishi.tianqi.com/baoanqu/201702.html', '20', '15','小雨','北风','2级']
,['2017-08-23','迁安市', '唐山市', '河北省', 'http://lishi.tianqi.com/qianan1/201708.html', '29', '20', '小雨', '东风', '2级']
,['2017-08-23', '寿光市', '潍坊市', '山东省', 'http://lishi.tianqi.com/shouguang/201708.html', '31', '26', '雷阵雨','北风','1级']
,['2017-02-16','浦口区','南京市', '江苏省', 'http://lishi.tianqi.com/pukou/201702.html', '17', '8', '小雨', '西南风', '2级']

,['2017-02-23', '湖滨区', '三门峡市','河南省','http://lishi.tianqi.com/hubinqu/201702.html','9','-2','多云','西风','1级']
,['2017-08-24', '静海区', '[]', '天津市', 'http://lishi.tianqi.com/jinghai/201708.html', '32', '21', '晴', '北风', '2级']
,['2017-02-27', '东湖区', '南昌市', '江西省', 'http://lishi.tianqi.com/donghuqu/201702.html', '19', '9','晴','东风','2级'] 
,['2017-08-25','马村区','焦作市','河南省', 'http://lishi.tianqi.com/macunqu/201708.html', '29', '25', '小雨', '西风', '2级']
,['2017-03-03', '城中区', '西宁市', '青海省', 'http://lishi.tianqi.com/xining/201703.html','8','-9','多云','东南风','2级']
,['2017-08-26','蜀山区', '合肥市', '安徽省', 'http://lishi.tianqi.com/shushanqu/201708.html', '32', '24', '多云', '东北风', '1级']
,['2017-03-08', '颍州区', '阜阳市', '安徽省', 'http://lishi.tianqi.com/yingzhouqu/201703.html','15','2','晴','西北风', '1级']

,['2017-08-27','云龙区','徐州市','江苏省','http://lishi.tianqi.com/yunlongqu/201708.html', '30','25','多云','东南风','2级']

,['2017-03-28', '胶州市', '青岛市', '山东省', 'http://lishi.tianqi.com/jiaozhou/201703.html', '7', '1', '小到中雨', '东北风','4级']

,['2017-08-29','铜山区','徐州市','江苏省', 'http://lishi.tianqi.com/tongshan1/201708.html', '25', '17', '小雨', '北风', '2级']
,['2017-08-30', '彭山区','眉山市', '四川省', 'http://lishi.tianqi.com/pengshan/201704.html', '25', '21', '阴', '西风', '2级']

,['2017-04-07','阳城县', '晋城市', '山西省', 'http://lishi.tianqi.com/yangcheng/201704.html','21','12','多云','东南风','3级'] 
,['2017-08-30','阳曲县', '太原市', '山西省','http://lishi.tianqi.com/yangqu/201708.html', '25', '10', '晴', '西南风', '1级']

,['2017-09-01','成安县','邯郸市','河北省', 'http://lishi.tianqi.com/chengan/201709.html', '26','19','多云','南风','1级']
,['2017-04-10','广德县', '宣城市', '安徽省', 'http://lishi.tianqi.com/guangde/201704.html', '11', '8', '小雨', '西风', '4级']

,['2017-04-14', '黄岛区', '青岛市', '山东省', 'http://lishi.tianqi.com/huangdao/201704.html', '19', '12', '多云','南风','4级']

,['2017-09-01','成安县','邯郸市', '河北省', 'http://lishi.tianqi.com/chengan/201709.html', '26', '19', '多云', '南风', '1级']
,['2017-09-29', '东昌府区', '聊城市', '山东省', 'http://lishi.tianqi.com/liaocheng/201709.html','28','16','晴','南风','2级'] 

,['2017-09-29','乌拉特前旗', '巴彦淖尔市', '内蒙古自治区', 'http://lishi.tianqi.com/bayanzhuoer/201709.html', '24', '9', '晴', '西风', '1级']]



In [93]:
test_1.extend(test_2)
# test_1.extend(test_3)

In [96]:
test_1.extend(test_3)

In [98]:
len(test_1)

70

In [39]:
len(test_1)

11

In [101]:
res.extend(test_1)

In [102]:
len(res)

75562

In [44]:
res

['2017-06-27',
 '云龙区',
 '徐州市',
 '江苏省',
 'http://lishi.tianqi.com/yunlongqu/201706.html',
 '33',
 '24',
 '多云',
 '东风',
 '3级']

In [103]:
middle_res = []
for i in range(len(res)-1,-1,-1):
    test = res[i]
    if len(test)!=10:
        middle_res.append(test)
        res.pop(i)
#     print(len(res[i]))
    

In [104]:
len(res)

74993

In [107]:
middle_res

[['2017-07-19',
  '泰山区',
  '泰安市',
  '山东省',
  'http://lishi.tianqi.com/taishanqu/201707.html',
  '33',
  '26',
  '阴南风',
  '1级'],
 ['2017-12-27',
  '雨湖区',
  '湘潭市',
  '湖南省',
  'http://lishi.tianqi.com/yuhuqu/201712.html',
  '13',
  '7',
  '阴',
  '1级'],
 ['2017-10-25',
  '桥西区',
  '邢台市',
  '河北省',
  'http://lishi.tianqi.com/qiaoxiqu/201710.html',
  '16',
  '10',
  '小雨',
  '1级'],
 ['2017-10-24',
  '桥西区',
  '邢台市',
  '河北省',
  'http://lishi.tianqi.com/qiaoxiqu/201710.html',
  '17',
  '9',
  '晴',
  '1级'],
 ['2017-10-23',
  '桥西区',
  '邢台市',
  '河北省',
  'http://lishi.tianqi.com/qiaoxiqu/201710.html',
  '16',
  '7',
  '多云',
  '1级'],
 ['2017-10-22',
  '桥西区',
  '邢台市',
  '河北省',
  'http://lishi.tianqi.com/qiaoxiqu/201710.html',
  '16',
  '10',
  '多云',
  '2级'],
 ['2017-10-20',
  '桥西区',
  '石家庄市',
  '河北省',
  'http://lishi.tianqi.com/qiaoxiqu/201710.html',
  '22',
  '12',
  '晴',
  '2级'],
 ['2017-10-20',
  '桥西区',
  '邢台市',
  '河北省',
  'http://lishi.tianqi.com/qiaoxiqu/201710.html',
  '22',
  '12',
  '晴',
  '2级']

In [141]:
# clean_res = []
from copy import deepcopy
for i in range(len(middle_res)-1, -1, -1):
    middle_data = middle_res[i]
    if len(middle_data)==9:
        #检测风向
        sup_windirec = deepcopy(middle_data)
        if middle_data[7] in ['雾','晴','阴','多云','小雨','雨','阵雨','晴','雷阵雨','中雨','大雨','小到中雨','大到暴雨','小雪'] and (re.search("\d+级", middle_data[8]) or re.search("微风", middle_data[8])):
            sup_windirec.insert(8,'')
            clean_res.append(sup_windirec)
            middle_res.pop(i)
        

In [144]:
for i in middle_res:
    print(i)


['2017-07-19', '泰山区', '泰安市', '山东省', 'http://lishi.tianqi.com/taishanqu/201707.html', '33', '26', '阴南风', '1级']
['2017-09-03', '丰镇市', '乌兰察布市', '风', '2级']
['2017-04-24', '寒亭区', '潍坊市', '山东省', 'http://lishi.tianqi.com/hantingququ/201704.html', '25', '10', '雷阵雨', '北ml', '25', '15', '多云', '东南风', '1级']
['2017-09-02', '小店区', '太原市', '山西省', 'http://lishi.tianqi.com/xiaodianqu/201709.ht']
['2017-04-20', '闵行区', '[]', '上海市', 'http://lishi.tianqi.com/minhang/201704.html', '24', '14', '阴', '西风', '4级lishi.tianqi.com/fengnan/201709.html', '27', '18', '多云', '南风', '2级']
['2017-09-01', '长治县', '长治市', '山西省', 'http://', '南风', '4级']
['2017-04-10', '广德1', '当涂县', '马鞍山市', '安徽省', 'http://lishi.tianqi.com/dangtu/201708.html', '24', '21', '小雨', '东风', '4级']
['2017-08-3ishi.tianqi.com/yangcheng/201704.html', '21', '12', '多云', '东南风', '3级']
['2017-04-01', '彭山区7-08-30', '沁县', '长治市', '山西省', 'http://lishi.tianqi.com/qinxian/201708.html', '23', '11', '多云', '南风', '1级']
['2017-08-29', '铜山区', '徐州市', '江北风', '4级']
['2017-03-23',

In [156]:
cleaned_li = [['2017-07-19', '泰山区', '泰安市', '山东省', 'http://lishi.tianqi.com/taishanqu/201707.html', '33', '26', '阴','南风', '1级'],
['2017-09-03', '丰镇市', '乌兰察布市','内蒙古自治区','http://lishi.tianqi.com/fengzhen/201707.html','32','19','多云','西风', '2级'],
['2017-04-24', '寒亭区', '潍坊市', '山东省', 'http://lishi.tianqi.com/hantingququ/201704.html', '25', '10', '雷阵雨', '北风','2级'],    
['2017-09-02', '小店区', '太原市', '山西省', 'http://lishi.tianqi.com/xiaodianqu/201709.html', '25', '15', '多云', '东南风', '1级'],
['2017-04-20', '闵行区', '[]', '上海市', 'http://lishi.tianqi.com/minhang/201704.html', '24', '14', '阴', '西风', '4级'],
['2017-09-02','丰南区','唐山市','河北省','http://lishi.tianqi.com/fengnan/201709.html', '27', '18', '多云', '南风', '2级'],
['2017-09-01', '长治县', '长治市', '山西省', 'http://lishi.tianqi.com/changzhi/201709.html', '23','14','多云','南风','3级'],
['2017-04-10', '广德县', '宣城市','安徽省','http://lishi.tianqi.com/guangde/201704.html','11','8','小雨','西风','4级'],
['2017-08-31','当涂县', '马鞍山市', '安徽省', 'http://lishi.tianqi.com/dangtu/201708.html', '24', '21', '小雨', '东风', '4级'],
['2017-04-07','阳城县','晋城市','山西省','http://lishi.tianqi.com/yangcheng/201704.html', '21', '12', '多云', '东南风', '3级'],
['2017-04-01', '彭山区','眉山市','四川省','http://lishi.tianqi.com/pengshan/201704.html','24','15','多云','东北风','3级'],
['2017-08-30', '沁县', '长治市', '山西省', 'http://lishi.tianqi.com/qinxian/201708.html', '23', '11', '多云', '南风', '1级'],
['2017-08-29', '铜山区', '徐州市','江苏省','http://lishi.tianqi.com/tongshan1/201708.html','25','17','小雨','北风','2级'],
['2017-03-23', '武昌区', '武汉市', '湖北省', 'http://lishi.tianqi.com/wuchangqu/201703.html', '14', '9', '小雨','北风','4级'],
['2017-03-18', '成武县', '菏泽市', '山东省', 'http://lishi.tianqi.com/chengwu/201703.html','19','8','多云','东南风','1级'],
['2017-08-28','嘉祥县','济宁市', '山东省', 'http://lishi.tianqi.com/jiaxiang/201708.html', '27', '21', '阵雨', '北风', '2级'],
['2017-03-14', '江都区', '扬州市', '江苏省','http://lishi.tianqi.com/jiangdu/201703.html','13','0','晴','东北风','3级'],
['2017-08-27','复兴区','邯郸市','河北省', 'http://lishi.tianqi.com//fuxingqu/201708.html', '26', '19', '阴', '东北风', '3级'],
['2017-08-26', '壶关县', '长治市', '山西省', 'http://lishi.tianqi.com/huguan/201708.html', '21', '15', '小雨', '北风','2级'],
['2017-08-25', '马村区', '焦作市','河南省', 'http://lishi.tianqi.com/macunqu/201708.html','29','25','小雨','西风','2级'],
['2017-02-23','湖滨区','三门峡市','河南省', 'http://lishi.tianqi.com/hubinqu/201702.html', '9', '-2', '多云', '西风', '1级'],
['2017-02-16', '浦口区','南京市','江苏省','http://lishi.tianqi.com/pukou/201702.html','17','8','小雨','西南风','2级'],
['2017-08-24','黄骅市','沧州市','河北省', 'http://lishi.tianqi.com/huanghua/201708.html', '32', '22', '多云', '西风', '1级'],
['2017-02-12', '西华县', '周口市', '河南省', 'http://lishi.tianqi.com/xihua/201702.html','14','2','晴', '东南风', '2级'],
['2017-01-16', '西青区', '[]', '天津市', 'http://lishi.tianqi.com/xiqing/201701.html','1','-3','多云','东风','1级'],
['2017-08-22','曲阳县', '保定市', '河北省', 'http://lishi.tianqi.com/quyang/201708.html', '29', '21', '小雨', '北风', '1级'],
['2017-08-21', '金台区', '宝鸡市', '陕西省', 'http://lishi.tianqi.com/jintaiqu/201708.html', '26','20', '小雨', '西南风', '1级'],
['2017-01-06', '海州区', '连云港市', '江苏省', 'http://lishi.tianqi.com/haizhouqu/201701.html', '5','-9','霾','东北风','1级'],
['2017-12-26', '乌拉特前旗', '巴彦淖尔市','内蒙古自治区','http://lishi.tianqi.com/bayanzhuoer/201712.html','1','-12','晴','北风','2级'],
['2017-08-20', '东西湖区', '武汉市', '湖北省', 'http://lishi.tianqi.com/dongxihu/201708.html','33','26','阵雨','北风','1级'],
['2017-12-27','成安县','邯郸市', '河北省', 'http://lishi.tianqi.com/chengan/201712.html', '8', '-5', '晴', '北风', '1级'],
['2017-12-26', '江津区','[]', '重庆市', 'http://lishi.tianqi.com/jiangjin/201712.html', '11', '7', '多云', '东北风', '2级'],
['2017-08-19','滨城区','滨州市','山东省','http://lishi.tianqi.com/binxianqu/201708.html', '28', '23', '雷阵雨', '东北风', '2级'],
['2017-12-23', '安泽县', '临汾市', '山西省','http://lishi.tianqi.com/anze/201712.html','7','-8','阴','西风','2级'],
['2017-08-18','武侯区','成都市','四川省','http://lishi.tianqi.com/wuhouqu/201708.html', '29', '23', '小雨', '北风', '2级'],
['2017-12-19', '蔡甸区', '武汉市', '湖北省', 'http://lishi.tianqi.com/caidian/201712.html', '11','0','晴','西南风','2级'],
['2017-08-18', '武进区', '常州市', '江苏省', 'http://lishi.tianqi.com/wujin/201708.html', '34', '26', '雷阵雨', '南风','2级'],
['2017-12-15', '青山区', '武汉市', '湖北省', 'http://lishi.tianqi.com/qingshanqu/201712.html','-4','-13','晴','西北风','3级'],
['2017-08-17','玉田县','唐山市', '河北省', 'http://lishi.tianqi.com/yutian/201708.html', '30', '20', '晴', '西南风', '1级'],
['2017-12-12', '萧山区', '杭州市', '浙江省', 'http://lishi.tianqi.com/xiaoshan/201712.html', '12', '4', '多云', '东风', '4级'],
['2017-08-17', '河西区', '[]', '天津市', 'http://lishi.tianqi.com/hexiqu/201708.html', '27', '23', '多云','东南风', '1级'],
['2017-08-15', '天宁区', '常州市', '江苏省', 'http://lishi.tianqi.com/tianningqu/201708.html','30','25','多云','西风','2级'],
['2017-11-28', '交口县', '吕梁市', '山西省', 'http://lishi.tianqi.com/jiaokou/201711.html', '3', '-7', '多云', '西北风', '2级'],
['2017-08-14', '清江浦区', '淮安市', '江苏省', 'http://lishi.tianqi.com/huaian1/201708.html', '30','24','多云','西风','1级'],
['2017-11-23','常熟市','苏州市','江苏省','http:lishi.tianqi.com/changshu/201711.html', '12', '4', '多云', '西南风', '1级'],
['2017-08-14', '榆阳区', '榆林市', '陕西省', 'http://lishi.tianqi.com/yuyangqu/201708.html','29','18','多云','东南风','2级'],
['2017-08-12', '殷都区', '安阳市', '河南省', 'http://lishi.tianqi.com/yinduqu/201708.html', '30', '23', '阵雨', '南风', '1级'],
['2017-11-13','西工区', '洛阳市', '河南省', 'http://lishi.tianqi.com/xigongqu/201711.html', '17', '4', '晴', '西风', '2级'],
['2017-08-12', '泽州县', '晋城市', '山西省', 'http://lishi.tianqi.com/zezhou/201708.html', '30', '21', '雷阵雨', '东南风', '2级'],
['2017-11-10','滨城区','滨州市','','山东省', 'http://lishi.tianqi.com/binxianqu/201711.html', '13', '-1', '多云', '西北风', '3级'],
['2017-11-10', '滨城区', '滨州市','','http://lishi.tianqi.com/laoting/201708.html', '25', '0', '多云', '东南风', '小于3级'],
['2017-08-10', '宣化区', '张家口市', '河北省', 'http://lishi.tianqi.com/zhangjiakou/201708.html','33','19','多云','东南风','4级'],
['2017-11-04', '通州区', '[]','北京市','http://lishi.tianqi.com/tongzhou/201711.html','15','2','晴','东风','1级'],
['2017-08-10', '丰润区', '唐山市', '河北省', 'http://lishi.tianqi.com/fengrun/201708.html', '31', '25','多云','东风','2级'],
['2017-10-31', '永年区', '邯郸市', '河北省', 'http://lishi.tianqi.com/yongnian/201710.html', '19', '9', '多云', '南风', '3级'],
['2017-08-09', '长武县', '咸阳市', '陕西省', 'http://lishi.tianqi.com/changwu/201708.html', '29', '14', '晴','北风','2级'],
['2017-10-27','江宁区','南京市', '江苏省', 'http://lishi.tianqi.com/jiangning/201710.html', '11', '0', '晴', '东北风', '小于3级'],
['2017-08-08', '江宁区','临汾市', '山西省', 'http://lishi.tianqi.com/quwo/201708.html', '33', '19', '多云', '西北风', '1级'],
['2017-08-08', '曲沃县','临汾市','山西省','http://lishi.tianqi.com/quwo/201708.html','33','19','多云','西北风','1级'],
['2017-10-24','汝州市','平顶山市','河南省','http://tianqi.com/ruzhou/201710.html', '18', '12', '多云', '东北风', '1级'],
['2017-10-20', '管城回族区', '郑州市', '河南省', 'http://lishi.tianqi.com/guanchengqu/201710.html', '22','11','晴','东南风','1级'],
['2017-08-07', '秦都区', '咸阳市县',  '陕西省', 'http://lishi.tianqi.com/qinduqu/201708.html', '24', '20', '大雨', '西风', '2级'],
['2017-10-16','梁山县','济宁市', '山东省', 'http://lishi.tianqi.com/liangshan/201710.html', '19', '13', '阴', '北风', '1级'],
['2017-08-05', '鹤山区', '鹤壁市', '河南','http://lishi.tianqi.com/heshanqu1/201708.html','36','25','晴','','1级'],
['2017-10-05','丛台区', '邯郸市', '河北省','http://lishi.tianqi.com/congtaiqu/201710.html', '21', '12', '多云', '南风', '2级'],
['2017-08-04', '武安市', '邯郸市', '河北省', 'http://lishi.tianqi.com/wuan/201708.html', '36','25','晴','西风','2级'],
['2017-09-28','安阳县','安阳市', '河南省', 'http://lishi.tianqi.com/anyang/201709.html', '25', '16', '晴', '东南风', '3级'],
['2017-08-03', '新郑市', '郑州市', '河南省', 'http://lishi.tianqi.com/xinzheng/201708.html', '36','25','晴','南风','2级'],
['2017-09-24','丛台区', '邯郸市', '河北省', 'http://lishi.tianqi.com/congtaiqu/201709.html', '31', '19', '晴', '南风', '2级'],
['2017-09-20', '平山县', '石家庄市', '河北省', 'http://lishi.tianqi.com/pingshanqu/201709.html','24', '11', '晴', '西北风','2级'],
['2017-08-02', '曲阳县', '保定市', '河北省', 'http://lishi.tianqi.com/quyang/201708.html','30','23','雷阵雨','东风','2级'],
['2017-09-15', '城中区', '西宁市', '青海省', 'http://lishi.tianqi.com/chengzhong/201709.html', '32', '24', '多 云', '东北风', '2级'],
['2017-08-01', '栖霞区', '南京市', '江苏省', 'http://lishi.tianqi.com/qixia/201708.html', '31', '26', '雷阵雨', '东南风', '3级'],
['2017-09-11', '玉田县', '唐山市', '河北省', 'http://lishi.tianqi.com/yutian/201709.html', '28', '13', '晴', '西风', '1级'],
['2017-08-01', '交口县', '吕梁市', '山西省', 'http://lishi.tianqi.com/jiaokou/201708.html', '25', '17', '多云', '东南风', '1级'],
['2017-09-07', '乌拉特前旗', '巴彦淖尔市', '内蒙古自治区', 'http://lishi.tianqi.com/bayanzhuoer/201709.html', '31', '13', '晴', '西北风', '1级'], 
['2017-07-31','邯山区', '邯郸市', '河北省', 'http://lishi.tianqi.com/hanshanqu/201707.html', '31', '24', '晴', '东北风', '1级'],
['2017-09-03', '莱城区', '莱芜市', '山东省', 'http://lishi.tianqi.com/laiwu/201709.html', '28', '19', '多云','东风', '1级'],
['2017-07-30', '中方县', '怀化市', '湖南省', 'http://lishi.tianqi.com/zhongfang/201707.html', '36', '26', '多云','北风','2级'],
['2017-08-26', '莱城区', '莱芜市', '山东省','http://lishi.tianqi.com/laiwu/201708.html','30','21','多云','北风','微风'],
['2017-07-28','黄陵县','延安市','陕西省','http://lishi.tianqi.com/huangling/201707.html', '22', '18', '小雨', '北风', '1级'],
['2017-07-28', '路北区', '唐山市', '河北省', 'http://lishi.tianqi.com/tangshan/201707.html','31','21','多云','西南风','2级'],
['2017-08-19', '海港区', '秦皇岛市', '河北省', 'http://lishi.tianqi.com/haigangqu/201708.html', '28', '21', '阵雨', '东风', '1级'],
['2017-07-27', '岳阳楼区', '岳阳市', '湖南省', 'http://lishi.tianqi.com/yueyanglouqu/201707.html', '36', '31', '晴','东北风', '1级'],
['2017-07-26', '运河区', '沧州市', '河北省', 'http://lishi.tianqi.com/yunhequ/201707.html', '28', '22', '中雨', '北风', '2级'],
['2017-08-12','博兴县','滨州市','山东省','http://lishi.tianqi.com/boxing/201708.html', '34', '24', '多云', '东风', '2级'],
['2017-08-08', '巴南区', '[]', '重庆市', 'http://lishi.tianqi.com/banan/201708.html','26','22','大雨','西南风','2级'],
['2017-07-25','江油市', '绵阳市', '四川省', 'http://lishi.tianqi.com/jiangyou/201707.html', '32', '23', '小雨', '东风', '1级'],
['2017-07-28', '东港区', '日照市', '山东省', 'http://lishi.tianqi.com/donggang1/201707.html', '30', '21', '多云', '北风', '2级'],
['2017-07-23', '屯留县', '长治市', '山西省', 'http://lishi.tian qi.com/tunliu/201707.html', '33', '22', '多云', '北风', '4级'],
['2017 07-23', '江油市', '绵阳市', '四川省', 'http://lishi.tianqi.com/jiangyou/201707.html', '32', '23', '小雨', '东北风', '1级'],
['2017-07-19','泰山区','泰安市','山东省','http://lishi.tianqi.com/taishanqu/201707.html', '33', '26', '阴', '南风', '1级'],
['2017-07-15', '黄陵县', '延安市', '陕西省', 'http://lishi.tianqi.com/huangling/201707.html','35','20','小雨','西风','1级'],
['2017-07-12', '乌拉特前旗', '巴彦淖尔市', '内蒙古自治区', 'http://lishi.tianqi.com/bayanzhuoer/201707.html', '0', '0', '东风', '4-5级'],
['2017-07-21', '耀州区', '铜川市', '陕西省', 'http://lishi.tianqi.com/yaozhou/201707.html', '38', '24', '多云', '东'],
['2017-07-16', '新兴区', '七台河市', '黑龙江省','http://lishi.tianqi.com/xinxing/201707.html', '32','25','中雨','东风','2级'],
['2017-06-14', '静海区', '[]', '天津市', 'http://lishi.tianqi.com/jinghai/201706.html', '36','22','晴','西南风','3级'],
['2017-07-16', '彬县', '咸阳市', '陕西省', 'http://lishi.tianqi.com/binxian1/201707.html', '30', '20', '小雨', '东南风', '2级'],
['2017-06-10', '江宁区', '南京市', '江苏省', 'http://lishi.tianqi.com/jiang ning/201706.html', '25', '20', '大到暴雨', '东风', '3级'],
['2017-07-15', '武陟县', '焦作市', '河南省', 'http://lishi.tianqi.com/wuzhi/201707.html', '32', '24', '中雨', '东南风', '1级'],
['2017-06-06','耀州区', '铜川市','陕西省','http://lishi.tianqi.com/yaozhou/201706.html', '25', '15', '晴', '东北风', '5级']]

In [153]:
cleaned_1 = [['2017-07-13', '新兴区', '七台河市', '黑龙江省', 'http://lishi.tianqi.com/xinxing/201707.html', '34', '25', '多云', '北风', '1级'],
['2017-05-21', '复兴区', '邯郸市','河北省', 'http://lishi.tianqi.com/fuxingqu/201705.html', '32', '21', '晴', '东南风', '4级'],
['2017-07-12', '闻喜县', '运城市', '山西省', 'http://lishi.tianqi.com/wenxi/201707.html', '39', '24', '晴', '东', '南风', '2级'],
['2017-07-12', '乌拉特前旗', '巴彦淖尔市', '内蒙古自治区', 'http://lishi.tianqi.com/bayanzhuoer/201707.html', '0', '0','', '东风', '4-5级'],
['2017-05-16', '襄汾县', '临汾市', '山西省', '山东省', 'http://lishi.tianqi.com/xiangfen/201705.html', '32', '15', '晴', '南风', '5级'],
['2017-07-12', '寿光市', '潍坊市', 'http://lishi.tianqi.com/shouguang/201707.html', '37', '27', '晴', '南风', '2级'],
['2017-05-10', '丛台区', '邯郸市', '河北省', 'http://lishi.tianqi.com/congtaiqu/201705.html', '32', '19', '晴', '东南风', '3级'],
['2017-07-11','岳阳楼区', '岳阳市', '湖南省', 'http://lishi.tianqi.com/yueyanglouqu/201707.html', '30', '27', '多云', '南风', '2级'],
['2017-07-10', '玄武区', '南京市', '江苏省', 'http://lishi.tianqi.com/xuanwuqu/201707.html', '29', '24', '阴', '西南风', '2级'],
['2017-04-19', '滦县', '唐山市', '河北省', 'http://lishi.tianqi.com/luanxian/index.html', '20','9','阴','东南风','1级']]




In [157]:
cleaned_li.extend(cleaned_1)

In [159]:
res.extend(clean_res)

In [160]:
res.extend(cleaned_li)

In [161]:
len(res)

75579

In [166]:
# middle_res = []
for i in range(len(res)-1,-1,-1):
    test = res[i]
    if len(test)!=10:
        print(test)
#         middle_res.append(test)
        res.pop(i)
#     print(len(res[i]))

In [164]:
clean_2 = [
    ['2017-07-12', '寿光市', '潍坊市','山东省', 'http://lishi.tianqi.com/shouguang/201707.html', '37', '27', '晴', '南风', '2级'],
    ['2017-05-16', '襄汾县', '临汾市', '山西省', 'http://lishi.tianqi.com/xiangfen/201705.html', '32', '15', '晴', '南风', '5级'],
    ['2017-07-12', '闻喜县', '运城市', '山西省', 'http://lishi.tianqi.com/wenxi/201707.html', '39', '24', '晴', '东南风', '2级'],
    ['2017-07-21', '耀州区', '铜川市', '陕西省', 'http://lishi.tianqi.com/yaozhou/201707.html', '38', '24', '多云', '东风','2级'],
    ['2017-07-12', '乌拉特前旗', '巴彦淖尔市', '内蒙古自治区', 'http://lishi.tianqi.com/bayanzhuoer/201707.html', '0', '0','','东风', '4-5级'],
    ['2017-11-10', '滨城区', '滨州市', '山东省', 'http://lishi.tianqi.com/binxianqu/201711.html', '13', '-1', '多云', '西北风', '3级']



]

In [165]:
res.extend(clean_2)

In [280]:
weather_df = pd.DataFrame(res, columns = ['日期','district','city','province','url','max_temp','min_temp','weather','wind_direct','wind_scale'])

In [281]:
weather_df.head()

,日期,district,city,province,url,max_temp,min_temp,weather,wind_direct,wind_scale
0,2017-01-01,江岸区,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13,4,多云,北风,微风
1,2017-01-01,宜秀区,安庆市,安徽省,http://lishi.tianqi.com/yixiuqu/201701.html,15,4,多云,东南风,1级
2,2017-01-01,萧山区,杭州市,浙江省,http://lishi.tianqi.com/xiaoshan/201701.html,16,8,晴,北风,3级
3,2017-01-01,中原区,郑州市,河南省,http://lishi.tianqi.com/zhongyuanqu/201701.html,7,0,霾,西风,1级
4,2017-01-01,宿城区,宿迁市,江苏省,http://lishi.tianqi.com/suchengqu/201701.html,12,4,多云,东风,1级


In [282]:
writer = pd.ExcelWriter("天气数据_1.xlsx")
weather_df.to_excel(writer, "sheet1")
writer.save()

In [50]:
loc_detail = pd.read_excel("定价系统训练数据_地区_detail.xlsx")
weather_df = pd.read_excel("天气数据.xlsx")
weather_df_1 = pd.read_excel("天气数据_1.xlsx")

In [77]:
weather_df_total = pd.concat([weather_df, weather_df_1],ignore_index = True)

In [80]:
len(weather_df_total)

70439

In [79]:
weather_df_total = weather_df_total.drop_duplicates(["日期", "district"])

In [65]:
weather_df_total.loc[71750:71781]

,日期,district,city,province,url,max_temp,min_temp,weather,wind_direct,wind_scale
71750,2017-12-14,[],东莞市,广东省,http://lishi.tianqi.com/dongguan/201712.html,20,16,阴,北风,2级
71751,2017-12-14,江都区,扬州市,江苏省,http://lishi.tianqi.com/jiangdu/201712.html,6,4,小雨,东风,2级
71752,2017-12-14,姑苏区,苏州市,江苏省,http://lishi.tianqi.com/suzhou/201712.html,8,6,阴,东北风,2级
71753,2017-12-14,九原区,包头市,内蒙古自治区,http://lishi.tianqi.com/jiuyuanqu/201712.html,1,-13,雨夹雪,北风,3级
71754,2017-12-14,金乡县,济宁市,山东省,http://lishi.tianqi.com/jinxiang/201712.html,4,-1,小雪,东风,1级
71755,2017-12-14,武昌区,武汉市,湖北省,http://lishi.tianqi.com/wuchangqu/201712.html,7,3,小雨,北风,1级
71756,2017-12-14,同安区,厦门市,福建省,http://lishi.tianqi.com/tongan/201712.html,21,16,多云,东南风,1级
71757,2017-12-14,虞城县,商丘市,河南省,http://lishi.tianqi.com/yucheng1/201712.html,4,0,雨夹雪,北风,1级
71758,2017-12-14,准格尔旗,鄂尔多斯市,内蒙古自治区,http://lishi.tianqi.com/zhungeer/201712.html,0,-11,小雪,西风,1级
71759,2017-12-14,昆山市,苏州市,江苏省,http://lishi.tianqi.com/kunshan/201712.html,9,6,小雨,东北风,1级


In [66]:
weather_df_total['max_temp'] = weather_df_total['max_temp'].astype(int)
weather_df_total['min_temp'] = weather_df_total['min_temp'].astype(int)

In [49]:
weather_df_total = weather_df_total.drop_duplicates()
len(weather_df_total)

71780

In [81]:
weather_df_total.head()

,日期,district,city,province,url,max_temp,min_temp,weather,wind_direct,wind_scale
0,2017-01-01,武安市,邯郸市,河北省,http://lishi.tianqi.com/wuan/201701.html,6,-2,雾,南风,1级
1,2017-01-01,玉田县,唐山市,河北省,http://lishi.tianqi.com/yutian/201701.html,-1,-7,雾,西南风,1级
2,2017-01-01,河西区,[],天津市,http://lishi.tianqi.com/hexiqu/201701.html,3,-2,雾,西风,2级
3,2017-01-01,武陟县,焦作市,河南省,http://lishi.tianqi.com/wuzhi/201701.html,9,0,多云,西南风,1级
4,2017-01-01,河东区,临沂市,山东省,http://lishi.tianqi.com/hedongqu/201701.html,4,-2,雾,西南风,1级


In [44]:
loc_detail = loc_detail.drop_duplicates()

In [297]:
loc_detail.head()

,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,目的地纬度,...,start_formatted_address,start_district,start_city,start_province,start_township,desti_formatted_address,desti_district,desti_city,desti_province,desti_township
0,2017-01-01,1,148,0,1,3333.33,114.203697,36.696506,114.670032,36.444317,...,河北省邯郸市武安市武安镇武安市人民政府,武安市,邯郸市,河北省,武安镇,河北省邯郸市成安县成安镇河北省成安县政协,成安县,邯郸市,河北省,成安镇
1,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,30.600064,...,河北省唐山市玉田县玉田镇玉田县人大常委会,玉田县,唐山市,河北省,玉田镇,湖北省武汉市江岸区四唯街街道武汉市江岸区人民政府江岸区四唯街胜江社区,江岸区,武汉市,湖北省,四唯街街道
2,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,34.341568,...,天津市河西区友谊路街道天津市委宣传部天津市人民政府,河西区,[],天津市,友谊路街道,陕西省西安市未央区张家堡街道西安市林业局西安市人民政府,未央区,西安市,陕西省,张家堡街道
3,2017-01-01,2,225,0,1,8064.52,113.401679,35.099378,117.063754,30.543494,...,河南省焦作市武陟县木城街道兴华路8号,武陟县,焦作市,河南省,木城街道,安徽省安庆市宜秀区菱北街道湖心北路84号安庆广播电视中心,宜秀区,安庆市,安徽省,菱北街道
4,2017-01-01,1,135,0,1,3672.87,118.402893,35.089917,118.909246,32.096228,...,山东省临沂市河东区九曲街道河东区,河东区,临沂市,山东省,九曲街道,江苏省南京市栖霞区仙林街道南京市栖霞区人民政府,栖霞区,南京市,江苏省,仙林街道


In [19]:
weather_df_total.head()

,日期,district,city,province,url,max_temp,min_temp,weather,wind_direct,wind_scale
0,2017-01-01,江岸区,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13,4,多云,北风,微风
1,2017-01-01,宜秀区,安庆市,安徽省,http://lishi.tianqi.com/yixiuqu/201701.html,15,4,多云,东南风,1级
2,2017-01-01,萧山区,杭州市,浙江省,http://lishi.tianqi.com/xiaoshan/201701.html,16,8,晴,北风,3级
3,2017-01-01,中原区,郑州市,河南省,http://lishi.tianqi.com/zhongyuanqu/201701.html,7,0,霾,西风,1级
4,2017-01-01,宿城区,宿迁市,江苏省,http://lishi.tianqi.com/suchengqu/201701.html,12,4,多云,东风,1级


In [27]:
weather_df_total.head()

,日期,start_district,start_city,start_province,url,max_temp,min_temp,weather,wind_direct,wind_scale
0,2017-01-01,江岸区,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13,4,多云,北风,微风
1,2017-01-01,宜秀区,安庆市,安徽省,http://lishi.tianqi.com/yixiuqu/201701.html,15,4,多云,东南风,1级
2,2017-01-01,萧山区,杭州市,浙江省,http://lishi.tianqi.com/xiaoshan/201701.html,16,8,晴,北风,3级
3,2017-01-01,中原区,郑州市,河南省,http://lishi.tianqi.com/zhongyuanqu/201701.html,7,0,霾,西风,1级
4,2017-01-01,宿城区,宿迁市,江苏省,http://lishi.tianqi.com/suchengqu/201701.html,12,4,多云,东风,1级


In [82]:
detail_weather_loc = loc_detail.merge(weather_df_total, left_on = ['日期','start_district','start_city','start_province'], 
                                     right_on = ['日期','district','city','province'],how = "left")

In [84]:
detail_weather_loc = detail_weather_loc.rename(columns = {"url":"start_url","max_temp":"start_max_temp","min_temp":"start_min_temp","weather":"start_weather","wind_direct":"start_wind_direct","wind_scale":"start_wind_scale"})

In [83]:
print(len(detail_weather_loc), len(loc_detail))
print(detail_weather_loc.columns)

213645 213645
Index(['日期', '一级品类', '详细品类', '订单类型', '交易类型', '货值（吨/元）', '起运地经度', '起运地纬度',
       '目的地经度', '目的地纬度', '运距', '车型', '车长', '运价（元/吨）',
       'start_formatted_address', 'start_district', 'start_city',
       'start_province', 'start_township', 'desti_formatted_address',
       'desti_district', 'desti_city', 'desti_province', 'desti_township',
       'district', 'city', 'province', 'url', 'max_temp', 'min_temp',
       'weather', 'wind_direct', 'wind_scale'],
      dtype='object')


In [74]:
detail_weather_loc[215060:215063]


detail_weather_loc.to_csv("test.csv")

In [85]:
detail_weather_loc = detail_weather_loc.merge(weather_df_total, left_on = ['日期','desti_district','desti_province'], 
                                      right_on = ['日期','district','province'], how = "left")

In [86]:
detail_weather_loc = detail_weather_loc.rename(columns = {"url":"desti_url","max_temp":"desti_max_temp","min_temp":"desti_min_temp","weather":"desti_weather","wind_direct":"desti_wind_direct","wind_scale":"desti_wind_scale"})

In [87]:
len(detail_weather_loc)

213645

In [313]:
detail_weather_loc.head()

,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,目的地纬度,...,start_wind_scale,district_y,city_y,province_y,desti_url,desti_max_temp,desti_min_temp,desti_weather,desti_wind_direct,desti_wind_scale
0,2017-01-01,1,148,0,1,3333.33,114.203697,36.696506,114.670032,36.444317,...,1级,成安县,邯郸市,河北省,http://lishi.tianqi.com/chengan/201701.html,3,-2,雾,北风,1级
1,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,30.600064,...,1级,江岸区,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13,4,多云,北风,微风
2,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,30.600064,...,1级,江岸区,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13,4,多云,北风,微风
3,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,34.341568,...,2级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,34.341568,...,2级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
detail_weather_loc.to_csv("detail_weather_loc.csv")

In [194]:
detail_weather_loc.head()

,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,目的地纬度,...,start_wind_direct,start_wind_scale,district_y,city_y,desti_url,desti_max_temp,desti_min_temp,desti_weather,desti_wind_direct,desti_wind_scale
0,2017-01-01,1,148,0,1,3333.33,114.203697,36.696506,114.670032,36.444317,...,南风,1级,成安县,邯郸市,http://lishi.tianqi.com/chengan/201701.html,3,-2,雾,北风,1级
1,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,30.600064,...,西南风,1级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,34.341568,...,西风,2级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01,2,225,0,1,8064.52,113.401679,35.099378,117.063754,30.543494,...,西南风,1级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,1,135,0,1,3672.87,118.402893,35.089917,118.909246,32.096228,...,西南风,1级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
import numpy as np
supp_data = detail_weather_loc[detail_weather_loc['desti_url'].isnull()]
supp_data_1 = detail_weather_loc[detail_weather_loc["start_url"].isnull()]

In [288]:
second_crawl_desti = supp_data.loc[:,["日期",'desti_district','desti_city','desti_province'] ]

In [90]:
len(supp_data_1)

3725

In [289]:
second_crawl_start = supp_data_1.loc[:,["日期",'start_district','start_city','start_province']]

In [91]:
len(supp_data)

8989

In [ ]:
supp_data

In [290]:
sup_weather_data = supp_data.merge(weather_df, left_on = ['日期','desti_district','desti_province'], 
                                      right_on = ['日期','district','province'], how = "left")

In [291]:
sup_weather_data.head()

,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,目的地纬度,...,desti_wind_scale,district,city,province_y,url,max_temp,min_temp,weather,wind_direct,wind_scale
0,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,30.600064,...,NaN,江岸区,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13,4,多云,北风,微风
1,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,34.341568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,2,225,0,1,8064.52,113.401679,35.099378,117.063754,30.543494,...,NaN,宜秀区,安庆市,安徽省,http://lishi.tianqi.com/yixiuqu/201701.html,15,4,多云,东南风,1级
3,2017-01-01,1,135,0,1,3672.87,118.402893,35.089917,118.909246,32.096228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,1,135,0,1,4211.12,118.402893,35.089917,118.909246,32.096228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
second_crawl_desti = second_crawl_desti.rename(columns = {"desti_district":"district",
                                                          "desti_city":"city",
                                                          "desti_province":"province" })

In [219]:
second_crawl_start = second_crawl_start.rename(columns = {'start_district':"district",
                                                          'start_city':"city",
                                                          'start_province':"province"})

In [220]:
second_crawl = pd.concat([second_crawl_desti, second_crawl_start])

In [221]:
second_crawl.head()

,日期,district,city,province
1,2017-01-01,江岸区,武汉市,湖北省
2,2017-01-01,未央区,西安市,陕西省
3,2017-01-01,宜秀区,安庆市,安徽省
4,2017-01-01,栖霞区,南京市,江苏省
5,2017-01-01,栖霞区,南京市,江苏省


In [222]:
len(second_crawl)

161415

In [224]:
len(second_crawl_start)+len(second_crawl_desti)

161415

In [256]:
second_crawl = second_crawl.drop_duplicates()

In [257]:
len(second_crawl)

43696